# 🧙‍♂️ 04. 정규표현식 (Regular Expression): 텍스트 마법사

## 1. 개요
"이 문서에서 **이메일 주소**만 다 찾아줘!"  
"전화번호의 **중간 자리**를 모두 `*`로 바꿔줘!"

이런 복잡한 문자열 검색/수정을 마법처럼 한 방에 해결해주는 도구가 바로 **정규표현식(Regex)**입니다.
처음엔 외계어 같지만, 익혀두면 평생 쓰는 프로그래머의 필수 무기입니다.

---

## 2. 정규표현식 주문서 (문법 총정리)

이 표에 있는 기호(메타 문자)만 알면 90%는 해결됩니다!

| 기호 | 이름 | 설명 | 쉬운 예시 |
|:---:|:---:|---|---|
| **.** | 점 | **"아무거나 한 글자"** | `a.c` ➡️ `abc`, `axc` (O) |
| **?** | 물음표 | **"있을 수도, 없을 수도"** | `ab?c` ➡️ `abc`(있음), `ac`(없음) |
| **\*** | 별 | **"0개 이상 (없어도 됨)"** | `ab*c` ➡️ `ac`, `abc`, `abbbc` |
| **+** | 플러스 | **"1개 이상 (최소 1개)"** | `ab+c` ➡️ `abc`, `abbbc` (O), `ac` (X) |
| **^** | 삿갓 | **"이걸로 시작해야 함"** | `^a` ➡️ `abc` (O), `bca` (X) |
| **$** | 달러 | **"이걸로 끝나야 함"** | `c$` ➡️ `abc` (O), `ab` (X) |
| **[ ]** | 대괄호 | **"이 중 하나"** (후보) | `[abc]` ➡️ `a` 또는 `b` 또는 `c` |
| **[^ ]** | 부정 | **"이거 빼고 다"** | `[^0-9]` ➡️ 숫자가 아닌 것 |


### 파이썬 도구 불러오기
`re` (Regular Expression) 라이브러리를 사용합니다.

In [ ]:
import re  # 파이썬 정규표현식 모듈


### 3-1. `.` (아무 글자 하나)
- `a.c` : a와 c 사이에 뭐든 1글자가 있어야 합니다.

In [ ]:
# 패턴 만들기: a + (아무거나 1개) + c
regex = re.compile('a.c')

# 1. 매칭되는 경우
print(regex.search('abc'))   # b가 . 자리에 들어감 (성공!)
print(regex.search('a@c'))   # @가 . 자리에 들어감 (성공!)

# 2. 매칭 안 되는 경우
print(regex.search('ac'))    # 중간에 글자가 없음 (실패)
print(regex.search('abbc'))  # 중간에 글자가 2개임 (실패)

### 3-2. `?` (있거나 없거나)
- `ab?c` : b가 있어도 되고 없어도 됩니다.

In [ ]:
regex = re.compile('ab?c')

print(regex.search('ac'))    # b가 0개 (성공)
print(regex.search('abc'))   # b가 1개 (성공)
print(regex.search('abbc'))  # b가 2개 (실패! 0개나 1개여야 함)

### 3-3. `*` (0개 이상 무제한)
- `ab*c` : b가 없어도 되고, 수백 개여도 됩니다.

In [ ]:
regex = re.compile('ab*c')

print(regex.search('ac'))          # b가 0개 (성공)
print(regex.search('abc'))         # b가 1개 (성공)
print(regex.search('abbbbbbbbc'))  # b가 많아도 (성공)

### 3-4. `+` (최소 1개 이상)
- `ab+c` : b가 **무조건 하나는** 있어야 합니다.

In [ ]:
regex = re.compile('ab+c')

print(regex.search('ac'))    # b가 0개 (실패!)
print(regex.search('abc'))   # b가 1개 (성공)
print(regex.search('abbc'))  # b가 2개 (성공)

### 3-5. `{n}` (횟수 지정)
- `ab{2}c` : b가 정확히 2번 나와야 합니다.

In [ ]:
regex = re.compile('ab{2}c')

print(regex.search('abc'))   # b가 1개 (실패)
print(regex.search('abbc'))  # b가 2개 (성공)
print(regex.search('abbbc')) # b가 3개 (실패)

### 3-6. `[]` (문자 후보 선택)
가장 많이 쓰입니다! 괄호 안의 문자들 중 **하나**와 매칭됩니다.

- `[abc]` : a, b, c 중 하나
- `[0-9]` : 숫자 전체 (0부터 9까지)
- `[a-z]` : 영어 소문자 전체
- `[가-힣]` : 한글 전체

In [ ]:
# 소문자(a-z) 중 하나라도 있으면 찾기
regex = re.compile('[a-z]')

print(regex.search('ABC'))         # 대문자만 있음 (실패)
print(regex.search('ABCdEF'))      # 소문자 d가 있음 (성공)

# 한글 찾기
korean_regex = re.compile('[가-힣]')
print(korean_regex.search('Hello'))     # 한글 없음 (실패)
print(korean_regex.search('Hello, 안녕')) # '안'에서 매칭 (성공)

## 4. 실전 예제: 전화번호와 숫자 추출하기

실무에서는 이런 식으로 많이 씁니다.
- `\d` : 숫자(Digit)를 의미합니다. `[0-9]`와 같습니다.

In [ ]:
text = "제 번호는 010-1234-5678 입니다. 나이는 25살이고요."

# 1. 숫자만 싹 다 뽑고 싶을 때
# \d+ : 숫자가 연속으로 붙어있는 덩어리들
numbers = re.findall('\d+', text)
print("숫자 추출:", numbers)

# 2. 전화번호 패턴만 뽑고 싶을 때
# 숫자들-숫자들-숫자들 형태
phones = re.findall('\d+-\d+-\d+', text)
print("전화번호 추출:", phones)

## 5. `re.sub()` : 청소하기 (치환)

특수문자를 지우거나 개인정보를 가릴 때 쓰는 **"찾아 바꾸기"** 기능입니다.

In [ ]:
text = "Hello!!! (World) ... >> ^^"

# 특수문자가 너무 많아서 지우고 싶습니다.
# [^a-zA-Z ] : 영어나 공백이 '아닌(^)' 모든 것
clean_text = re.sub("[^a-zA-Z ]", "", text)

print("원본:", text)
print("청소 후:", clean_text)

## 마무리

정규표현식은 `.` `*` `+` `?` `[]` 이 5가지 기호의 조합이 핵심입니다.
복잡해 보여도 이 기호들의 블록 조립이라고 생각하면 쉽습니다!